# Previsão Inadimplência

O Banco UniFinance, uma instituição financeira líder e confiável, destaca-se no
mercado pela sua dedicação em fornecer soluções de crédito acessíveis e sob
medida para empresários do setor comercial. Com foco em empréstimos
flexíveis e acessíveis, nossa equipe altamente qualificada trabalha em estreita
colaboração com os clientes para atender às suas necessidades financeiras
específicas.
Atualmente o banco está passando por uma revisão em como ele empresta o
dinheiro para os seus clientes, assim o objetivo é criar processos inteligentes
para a previsão de que alguém pode vim a passar por dificuldades financeiras
nos próximos dois anos.



**Entendimento do negócio**

No Banco UniFinance, quando um cliente solicita um empréstimo, iniciamos um
processo de avaliação que inclui a análise de diversos fatores, um dos quais é a possível ocorrência de dificuldades financeiras nos próximos dois anos. Isso é
crucial para identificar e mitigar riscos que poderiam levar à inadimplência, o
que, por sua vez, afetaria negativamente o banco. Nossa prioridade é garantir
empréstimos responsáveis e sustentáveis, tanto para o benefício do cliente
quanto para a segurança financeira da instituição.

# 0.0. IMPORTS

In [1]:
import pandas as pd

from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectPercentile
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score

import pickle

In [48]:
data = pd.read_csv("../data/train.csv", index_col=0)

# 1.0. DESCRIÇÃO DOS DADOS

In [49]:
df1 = data.copy()

## 1.1. Tamanho dos Dados

In [50]:
df1.shape

(150000, 11)

## 1.2. Nome das Colunas

In [5]:
df1.columns

Index(['target', 'TaxaDeUtilizacaoDeLinhasNaoGarantidas', 'Idade',
       'NumeroDeVezes30-59DiasAtrasoNaoPior', 'TaxaDeEndividamento',
       'RendaMensal', 'NumeroDeLinhasDeCreditoEEmprestimosAbertos',
       'NumeroDeVezes90DiasAtraso', 'NumeroDeEmprestimosOuLinhasImobiliarias',
       'NumeroDeVezes60-89DiasAtrasoNaoPior', 'NumeroDeDependentes'],
      dtype='object')

## 1.3. Tipo dos Dados

In [6]:
df1.dtypes

target                                          int64
TaxaDeUtilizacaoDeLinhasNaoGarantidas         float64
Idade                                           int64
NumeroDeVezes30-59DiasAtrasoNaoPior             int64
TaxaDeEndividamento                           float64
RendaMensal                                   float64
NumeroDeLinhasDeCreditoEEmprestimosAbertos      int64
NumeroDeVezes90DiasAtraso                       int64
NumeroDeEmprestimosOuLinhasImobiliarias         int64
NumeroDeVezes60-89DiasAtrasoNaoPior             int64
NumeroDeDependentes                           float64
dtype: object

## 1.4. Dados Faltantes

In [25]:
df1.isna().sum()

target                                            0
TaxaDeUtilizacaoDeLinhasNaoGarantidas             0
Idade                                             0
NumeroDeVezes30-59DiasAtrasoNaoPior               0
TaxaDeEndividamento                               0
RendaMensal                                   29731
NumeroDeLinhasDeCreditoEEmprestimosAbertos        0
NumeroDeVezes90DiasAtraso                         0
NumeroDeEmprestimosOuLinhasImobiliarias           0
NumeroDeVezes60-89DiasAtrasoNaoPior               0
NumeroDeDependentes                            3924
dtype: int64

## 1.5. Análise Estatística

In [14]:
df1.describe()

,target,TaxaDeUtilizacaoDeLinhasNaoGarantidas,Idade,NumeroDeVezes30-59DiasAtrasoNaoPior,TaxaDeEndividamento,RendaMensal,NumeroDeLinhasDeCreditoEEmprestimosAbertos,NumeroDeVezes90DiasAtraso,NumeroDeEmprestimosOuLinhasImobiliarias,NumeroDeVezes60-89DiasAtrasoNaoPior,NumeroDeDependentes
count,150000.000000,150000.000000,150000.000000,150000.000000,150000.000000,1.202690e+05,150000.000000,150000.000000,150000.000000,150000.000000,146076.000000
mean,0.066840,6.048438,52.295207,0.421033,353.005076,6.670221e+03,8.452760,0.265973,1.018240,0.240387,0.757222
std,0.249746,249.755371,14.771866,4.192781,2037.818523,1.438467e+04,5.145951,4.169304,1.129771,4.155179,1.115086
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.029867,41.000000,0.000000,0.175074,3.400000e+03,5.000000,0.000000,0.000000,0.000000,0.000000
50%,0.000000,0.154181,52.000000,0.000000,0.366508,5.400000e+03,8.000000,0.000000,1.000000,0.000000,0.000000
75%,0.000000,0.559046,63.000000,0.000000,0.868254,8.249000e+03,11.000000,0.000000,2.000000,0.000000,1.000000
max,1.000000,50708.000000,109.000000,98.000000,329664.000000,3.008750e+06,58.000000,98.000000,54.000000,98.000000,20.000000


In [15]:
df1['target'].value_counts(normalize=True)

target
0    0.93316
1    0.06684
Name: proportion, dtype: float64

## Separação dos dados de validação

In [51]:
X = df1.drop('target', axis=1)
y = df1.target

X_train, X_validation, y_train, y_validation = train_test_split(X, y, test_size=0.2, stratify=y)

In [52]:
imp_mean = SimpleImputer(strategy='mean')
cols_to_impute = ['RendaMensal', 'NumeroDeDependentes']
imp_mean.fit(X_train[cols_to_impute])

,missing_values,nan
,strategy,'mean'
,fill_value,None
,copy,True
,add_indicator,False
,keep_empty_features,False


In [53]:
X_train[cols_to_impute] = imp_mean.transform(X_train[cols_to_impute])
X_validation[cols_to_impute] = imp_mean.transform(X_validation[cols_to_impute])

## Seleção de Features

In [54]:
select = SelectPercentile(percentile=50)
select.fit(X_train, y_train)
# Transform training set
x_train_selected = select.transform(X_train)
print(f"X_train.shape: {X_train.shape}")
print(f"X_train_selected.shape: {x_train_selected.shape}")

X_train.shape: (120000, 10)
X_train_selected.shape: (120000, 5)


In [55]:
colunas_selecionadas = [X_train.columns[i] for i in range(len(X_train.columns[select.get_support()]))]
colunas_selecionadas

['TaxaDeUtilizacaoDeLinhasNaoGarantidas',
 'Idade',
 'NumeroDeVezes30-59DiasAtrasoNaoPior',
 'TaxaDeEndividamento',
 'RendaMensal']

## Treinar Modelo Baseline

In [56]:
x_valid_selected = select.transform(X_validation)

In [57]:
lr = LogisticRegression(max_iter=1000)
lr.fit(X_train, y_train)
y_hat = lr.predict_proba(X_validation)[:,1]

c:\Users\Pedro\repos\previsao_inadimplencia\mlflow\Lib\site-packages\sklearn\linear_model\_logistic.py:473: ConvergenceWarning: lbfgs failed to converge after 1000 iteration(s) (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT

Increase the number of iterations to improve the convergence (max_iter=1000).
You might also want to scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [58]:
lr.fit(x_train_selected, y_train)
y_hat_selected = lr.predict_proba(x_valid_selected)[:,1]

In [59]:
print(f"Score com TODAS as colunas: {round(roc_auc_score(y_validation, y_hat), 4)}")
print(f"Score com 50% das colunas: {round(roc_auc_score(y_validation, y_hat_selected), 4)}")

Score com TODAS as colunas: 0.7017
Score com 50% das colunas: 0.6988


## Tunning Hyperparameters

In [60]:
clf = [
    LogisticRegression(solver='newton-cg', penalty=None, max_iter=1000),
    LogisticRegression(solver='lbfgs', penalty=None, max_iter=1000),
    LogisticRegression(solver='sag', penalty=None, max_iter=1000),
    LogisticRegression(solver='saga', penalty=None, max_iter=1000)
]

clf_columns = []
clf_compare = pd.DataFrame(columns=clf_columns)

row_index = 0

for lrs in clf:
    y_hat = lrs.fit(X_train, y_train).predict_proba(X_validation)[:,1]
    auc = roc_auc_score(y_validation, y_hat)
    clf_name = lrs.__class__.__name__
    clf_compare.loc[row_index, "Modelo"] = clf_name
    clf_compare.loc[row_index, "max_iter"] = lrs.max_iter
    clf_compare.loc[row_index, "solver"] = lrs.solver
    clf_compare.loc[row_index, "penalty"] = lrs.penalty
    clf_compare.loc[row_index, "class_weight"] = lrs.class_weight
    clf_compare.loc[row_index, "AUC"] = auc

    row_index+=1

clf_compare.sort_values(by=["AUC"], ascending=False, inplace=True)
clf_compare

c:\Users\Pedro\repos\previsao_inadimplencia\mlflow\Lib\site-packages\sklearn\linear_model\_logistic.py:473: ConvergenceWarning: lbfgs failed to converge after 1000 iteration(s) (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT

Increase the number of iterations to improve the convergence (max_iter=1000).
You might also want to scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
c:\Users\Pedro\repos\previsao_inadimplencia\mlflow\Lib\site-packages\sklearn\linear_model\_sag.py:348: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
c:\Users\Pedro\repos\previsao_inadimplencia\mlflow\Lib\site-packages\sklearn\linear_model\_sag.py:348: ConvergenceWarning: The max_iter was reached which means the coef_ did not co

,Modelo,max_iter,solver,penalty,class_weight,AUC
0,LogisticRegression,1000.0,newton-cg,None,None,0.703247
1,LogisticRegression,1000.0,lbfgs,None,None,0.701446
2,LogisticRegression,1000.0,sag,None,None,0.578248
3,LogisticRegression,1000.0,saga,None,None,0.578029


## Deploy do Modelo

In [61]:
modelo_final = LogisticRegression(max_iter=1000,
                                  solver='newton-cg',
                                  penalty=None,
                                  class_weight=None
)

modelo_final.fit(X_train, y_train)

,penalty,None
,dual,False
,tol,0.0001
,C,1.0
,fit_intercept,True
,intercept_scaling,1
,class_weight,None
,random_state,None
,solver,'newton-cg'
,max_iter,1000
,multi_class,'deprecated'


In [ ]:
pickle.dump(modelo_final, open('modelo_final.pkl', 'wb'))

# 2.0. CRIAÇÃO DE FEATURES

## 2.1. Mapa Mental de Hipóteses

## 2.2. Novas Features

# 3.0. SELEÇÃO DE LINHAS E COLUNAS

# 4.0. ANÁLISE EXPLORATÓRIA

## 4.1. Análise Univariada

## 4.2. Análise Bivariada

## 4.3. Análise Multivariada

# 5.0. PREPARAÇÃO DOS DADOS

## Separação em treino e teste

## 5.1. Transformações

## 5.2. Encoders

# 6.0. SELEÇÃO DE FEATURES

# 7.0. ALGORITMOS DE MACHINE LEARNING

## 7.1. Algoritmo 1

### 7.1.1. Métrica de Avaliação

### 7.1.2. Validação Cruzada

# 8.0. AJUSTE DE HIPER-PARÂMETROS